In [1]:
import quantining.base as qt
import torch
import faiss
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/Users/marcindabrowski/conda/envs/3.11-research/lib/python3.11/site-packages/quantining/base/engine.py:15: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [43]:
from typing import Tuple, List, Literal

class BracketAccess(type):
    def __getitem__(cls, key: str):
        return getattr(cls, key, None)

class SampleMethods(metaclass=BracketAccess):

    @staticmethod
    def noise(scale: float, num_time_steps: int, **kwargs) -> np.ndarray:
        return np.random.normal(scale=scale, size=num_time_steps)

    @staticmethod
    def brownian_motion(num_time_steps: int, initial_value: int, drift=0.0, 
                                volatility=1.0, 
                                dt=1.0, **kwargs):
                                    
        increments = np.random.normal(loc=drift*dt, scale=volatility*np.sqrt(dt), size=num_time_steps)
    
        # Generate forward Brownian motion path
        path = np.cumsum(increments) + initial_value
    
        return path

    @staticmethod
    def random_oscillator(uniform_range: Tuple[int], num_time_steps: int, **kwargs) -> np.ndarray:
        return np.cos(np.random.uniform(*uniform_range, num_time_steps))

    @staticmethod
    def standardize(input_array: np.ndarray) -> np.ndarray:
        return (input_array - input_array.mean())/input_array.std()

    @staticmethod
    def normalize(input_array: np.ndarray) -> np.ndarray:
        return (input_array - input_array.min())/(input_array.max()-input_array.min())

    @staticmethod
    def random_dataset(n_series: int, 
                       series_types: List[Literal['noise', 'brownian_motion', 'random_oscillator']], 
                       **kwargs) -> pd.DataFrame:

        assert len(series_types) == n_series, AssertionError('len(series_types) must be equal to n_series')
        for name in series_types:
            assert name in ['noise', 'brownian_motion', 'random_oscillator'], AssertionError(f"{name} must be one of ['noise', 'brownian_motion', 'ranom_oscillator']")
        
        dataset = {}
                           
        for a, name in enumerate(series_types):
            data = SampleMethods[name](**kwargs)

            if 'transform' in kwargs.keys():
                match kwargs['transform']:
                    case 'normalize':
                        data = SampleMethods.normalize(data)
                    case 'standardize':
                        data = SampleMethods.standardize(data)
                    case other:
                        pass
                        
            dataset[f"{name}_{a}"] = data

        return pd.DataFrame(dataset)

    @staticmethod
    def all():
        return [key for key in SampleMethods.__dict__.keys() if not key.startswith('_')][:-1]

                           
    

In [51]:
class WindowTransform(metaclass=BracketAccess):

    @staticmethod
    def sliding_window(df: pd.DataFrame, window_length: int):
        sw = np.squeeze(np.lib.stride_tricks.sliding_window_view(df.values, (window_length,df.shape[-1])))
        return np.swapaxes(sw, 1,-1)

    @staticmethod
    def non_overlapping_window(df: pd.DataFrame, window_length: int):
        idxs = np.arange(df.shape[0]//window_length) * window_length
        stacked = np.dstack([df.iloc[idx:idx+window_length].values for idx in idxs])
        return np.swapaxes(stacked, 0,-1)

    @staticmethod
    def all():
        return [key for key in WindowTransform.__dict__.keys() if not key.startswith('_')][:-1]

    
        
        

In [16]:
from torch.utils.data import Dataset

In [258]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MultivariateDataset(Dataset):
    
    def __new__(cls, 
                mode: Literal['sliding_window', 'non_overlapping_window'], 
                df: pd.DataFrame,
                window_length: int, 
                transform=None):
        assert mode in WindowTransform.all(), AssertionError(f'mode must be on of {WindowTransform.methods()}')

        return super().__new__(cls)
        
    def __init__(self, 
                 mode: Literal['sliding_window', 'non_overlapping_window'], 
                 df: pd.DataFrame, 
                 window_length: int, 
                 transform=None):
                     
        global DEVICE
                     
        self.tensors = torch.FloatTensor(WindowTransform[mode](df, window_length)).to(DEVICE)  
        self.transform = transform

    def __len__(self):
        return len(self.tensors)

    def __getitem__(self, index):
        sample = self.tensors[index]

        if self.transform:
            sample = self.transofrm(sample)

        return sample.unsqueeze(0)

In [259]:
DF = SampleMethods.random_dataset(9, ['random_oscillator','random_oscillator','random_oscillator','brownian_motion','brownian_motion','brownian_motion','noise','noise','noise'], uniform_range=(0,1), num_time_steps=830*2880, initial_value=100, scale=0.5, transform='standardize')
SAMPLE_DATASET = MultivariateDataset('non_overlapping_window', DF, 2880)

In [260]:
SAMPLE_DATASET.tensors.shape

torch.Size([830, 9, 2880])

In [349]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.nn.modules.transformer import MultiheadAttention
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam

class Encoder(nn.Module):
    def __init__(self, input_dim, conv_filters, conv_kernel_size, conv_strides,
                 attention_heads, latent_dim):
        super(Encoder, self).__init__()

        # Initialize lists for convolutional layers
        self.conv_layers = nn.ModuleList()

        # Add convolutional layers
        for i in range(len(conv_filters)):
            self.add_conv_layer(input_dim if i==0 else conv_filters[i-1],
                                conv_filters[i], conv_kernel_size[i], conv_strides[i])

        # Multi-head self-attention mechanism
        self.self_attention = MultiheadAttention(embed_dim=conv_filters[-1], num_heads=attention_heads)

        # Fully connected layers to output the mean and standard deviation vectors
        self.fc_mu = nn.Linear(conv_filters[-1], latent_dim)
        self.fc_logvar = nn.Linear(conv_filters[-1], latent_dim)


    def add_conv_layer(self, in_channels, out_channels, kernel_size, stride):
        # Function to add a Convolutional layer followed by a ReLU activation
        self.conv_layers.append(nn.Conv1d(in_channels, out_channels, kernel_size, stride))
        self.conv_layers.append(nn.ReLU())

    def forward(self, x):

        # Pass through each Convolutional layer
        for layer in self.conv_layers:
            x = layer(x)

        # Store the output shape of the last Convolutional layer
        self.last_conv_output_shape = x.shape

        # Reshape x to match what the multi-head attention layer expects
        x = x.permute(2, 0, 1)  # shape becomes (L, N, E)

        # Apply self-attention
        x, _ = self.self_attention(x, x, x)

        # Fully connected layers to output the mean and standard deviation vectors
        x = x.mean(dim=0)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)

        return mu, logvar

    def reparameterize(self, mu, logvar):
        # Function to generate a random sample from the distribution defined by mu and logvar
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std


class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, conv_transpose_filters, conv_transpose_kernel_sizes, conv_transpose_strides, upsample):
        super(Decoder, self).__init__()

        # Fully connected layer
        self.fc = nn.Linear(latent_dim, hidden_dim)
        self.hidden_dim = hidden_dim

        # Upsample layer
        self.upsample = nn.Upsample(scale_factor=upsample)  # adjust this value as needed

        # Initialize list for Convolutional Transpose layers
        self.conv_transpose_layers = nn.ModuleList()

        # Add Convolutional Transpose layers
        for i in range(len(conv_transpose_filters)):
            self.add_conv_transpose_layer(hidden_dim if i==0 else conv_transpose_filters[i-1],
                                          conv_transpose_filters[i], conv_transpose_kernel_sizes[i], conv_transpose_strides[i])

    def add_conv_transpose_layer(self, in_channels, out_channels, kernel_size, stride):
        # Function to add a Convolutional Transpose layer followed by a ReLU activation
        self.conv_transpose_layers.append(nn.ConvTranspose1d(in_channels, out_channels, kernel_size, stride))
        self.conv_transpose_layers.append(nn.LeakyReLU())


    def forward(self, z):
        # Fully connected layer
        z = F.relu(self.fc(z))

        # Reshape to 3D tensor 
        z = z.view(-1, self.hidden_dim, 1)

        # Upsample
        z = self.upsample(z)

        # Pass through each Convolutional Transpose layer
        for layer in self.conv_transpose_layers:
            z = layer(z)

        return z

class HybridVAE(nn.Module):
    """
    Inspired by... with some additional changes
    """
    def __init__(self, input_dim, latent_dim, encoder_params, decoder_params):
        super(HybridVAE, self).__init__()

        self.encoder = Encoder(input_dim, *encoder_params, latent_dim)
        self.decoder = Decoder(latent_dim, *decoder_params)

    def forward(self, x):

        mu, logvar = self.encoder(x)
        z = self.encoder.reparameterize(mu, logvar)
        
        return self.decoder(z), mu, logvar

class Metrics(metaclass=BracketAccess):

    @staticmethod
    def mae(y_true, y_pred, **kwargs):
        return torch.mean(torch.abs(y_true-y_pred))

    @staticmethod
    def mse(y_true, y_pred, **kwargs):
        return torch.mean((y_true-y_pred)**2)

    @staticmethod
    def mase(y_true, y_pred, y_naive, **kwargs):
        mae = Metrics.mae(y_true, y_pred)
        scale = Metrics.mae(y_true, y_naive)
        return mae/scale

    def all():
        return [key for key in Metrics.__dict__.keys() if not key.startswith("_")][:-1]

In [313]:
from enum import Enum

Optimizers = Enum("Optimizers", {
 'Adadelta': torch.optim.Adadelta,
 'Adagrad': torch.optim.Adagrad,
 'Adam': torch.optim.Adam,
 'AdamW': torch.optim.AdamW,
 'SparseAdam': torch.optim.SparseAdam,
 'Adamax': torch.optim.Adamax,
 'ASGD': torch.optim.ASGD,
 'SGD': torch.optim.SGD,
 'RAdam': torch.optim.RAdam,
 'Rprop': torch.optim.Rprop,
 'RMSprop': torch.optim.RMSprop,
 'Optimizer': torch.optim.Optimizer,
 'NAdam': torch.optim.NAdam,
 'LBFGS': torch.optim.LBFGS,})

In [341]:
from typing import List, Sequence
from tqdm.autonotebook import tqdm

class Reducer:
    """
    
    """
    def __new__(cls, 
                dataset: MultivariateDataset, 
                batch_size: int,
                optimizer: Literal['Adadelta','Adagrad','Adam','AdamW','SparseAdam','Adamax','ASGD','SGD','RAdam','Rprop','RMSprop','Optimizer','NAdam','LBFGS'],
                latent_dim: int, 
                conv_filters: Sequence[int], 
                conv_kernel_size: Sequence[int], 
                conv_strides: Sequence[int], 
                attention_heads: int,
                hidden_dim: int, 
                conv_transpose_filters: Sequence[int], 
                conv_transpose_kernel_sizes:Sequence[int], 
                conv_transpose_strides:Sequence[int], 
                upsample: int):

        
        assert len(conv_filters) == len(conv_kernel_size) == len(conv_strides), AssertionError("All encoder arguments have to have same length")
        assert isinstance(dataset, MultivariateDataset), AssertionError("Dataset have to be of type MultivariateDataset")

        return super().__new__(cls)
                     
    def __init__(self, 
                 dataset: MultivariateDataset, 
                 batch_size: int,
                 optimizer: Literal['Adadelta','Adagrad','Adam','AdamW','SparseAdam','Adamax','ASGD','SGD','RAdam','Rprop','RMSprop','Optimizer','NAdam','LBFGS'],
                 latent_dim: int, 
                 conv_filters: Sequence[int], 
                 conv_kernel_size: Sequence[int], 
                 conv_strides: Sequence[int], 
                 attention_heads: int,
                 hidden_dim: int, 
                 conv_transpose_filters: Sequence[int], 
                 conv_transpose_kernel_sizes:Sequence[int], 
                 conv_transpose_strides:Sequence[int], 
                 upsample: int) -> None:
    
        global DEVICE

        self.data_loader = DataLoader(dataset, batch_size=batch_size)                      
        self.model = HybridVAE(input_dim=dataset.tensors.shape[1], 
                               latent_dim=latent_dim,
                               encoder_params=(conv_filters, conv_kernel_size, conv_strides, attention_heads),
                               decoder_params=(hidden_dim, conv_transpose_filters, 
                                               conv_transpose_kernel_sizes, conv_transpose_strides, 
                                               upsample)).to(DEVICE)
        self.optimizer = Optimizers[optimizer].value(params=self.model.parameters())

    @staticmethod
    def loss_hybrid_vae(recon_x, x, mu, logvar):
        
        # Reconstruction loss
        recon_loss = F.mse_loss(recon_x, x)
    
        # KL divergence loss
        kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
        # Total loss
        loss = recon_loss + kl_loss
    
        return loss
        
    def fit(self,
            epochs: int = 5, 
            metrics: Sequence[str] = None, schedule: bool = False,
            **kwargs) -> None:

        """
        Kwargs y_naive if metrics is mase
        """
        
        if metrics != None:
            assert hasattr(metrics, "__iter__"), AssertionError('If not none, metrics have to be iterable object')
            for name in metrics:
                assert name in Metrics.all(), AssertionError(f"{name} must be one of {Metrics.all()}.")
        
    
        desc = "Fitting VAE model on dataset..."
                
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min')
        #schedule = kwargs['schedule'] if 'schedule' in kwargs.keys() else None
                
        # Training loop
        for epoch in tqdm(range(epochs), total=epochs, desc=desc):
            summary = {'epoch':epoch+1}
            for batch in self.data_loader:
                x = batch[0]
                self.optimizer.zero_grad()
                recon_x, mu, logvar = self.model(x)
                loss = Reducer.loss_hybrid_vae(recon_x, x, mu, logvar)
                summary['loss'] = loss.item() #.:4f
                if metrics is not None:
                    for name in metrics:
                        summary[name] = Metrics[name](recon_x, x, **kwargs).item() #:.4f

                loss.backward()
                self.optimizer.step()
            print(summary)
            
            if schedule:
                scheduler.step(loss)

    def generate(self, **kwargs):
        pass

    def latent_rep(self, **kwargs) -> np.ndarray:

        encoded = []
        with torch.no_grad():
            for batch in tqdm(self.data_loader):
                x = batch[0]
                mu, logvar = self.model.encoder(x)
                z = self.model.encoder.reparameterize(mu, logvar)
                encoded.append(z)

        return np.vstack([item.cpu().numpy() for item in encoded])

In [344]:
CONV_FILTERS = [64,128]
EXAMPLE_PARAMS = {'dataset': SAMPLE_DATASET, 
                 'batch_size': 32,
                 'optimizer': 'Adam',
                 'latent_dim': 1000, 
                 'conv_filters': CONV_FILTERS,
                 'conv_kernel_size': [3,3], 
                 'conv_strides': [2,2],
                 'attention_heads':8,
                 'hidden_dim': 400, 
                 'conv_transpose_filters': [128,9], 
                 'conv_transpose_kernel_sizes': [2,2], 
                 'conv_transpose_strides': [2,2], 
                 'upsample': SAMPLE_DATASET.tensors.shape[-1]/(len(CONV_FILTERS)*2)}

In [345]:
r = Reducer(**EXAMPLE_PARAMS)

720.0 <class 'float'>


In [346]:
r.model

HybridVAE(
  (encoder): Encoder(
    (conv_layers): ModuleList(
      (0): Conv1d(9, 64, kernel_size=(3,), stride=(2,))
      (1): ReLU()
      (2): Conv1d(64, 128, kernel_size=(3,), stride=(2,))
      (3): ReLU()
    )
    (self_attention): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
    )
    (fc_mu): Linear(in_features=128, out_features=1000, bias=True)
    (fc_logvar): Linear(in_features=128, out_features=1000, bias=True)
  )
  (decoder): Decoder(
    (fc): Linear(in_features=1000, out_features=400, bias=True)
    (upsample): Upsample(scale_factor=720.0, mode='nearest')
    (conv_transpose_layers): ModuleList(
      (0): ConvTranspose1d(400, 128, kernel_size=(2,), stride=(2,))
      (1): LeakyReLU(negative_slope=0.01)
      (2): ConvTranspose1d(128, 9, kernel_size=(2,), stride=(2,))
      (3): LeakyReLU(negative_slope=0.01)
    )
  )
)

In [348]:
r.fit(epochs=20, metrics=['mae','mse'], schedule=True)

Fitting VAE model on dataset...:   5%|███▎                                                              | 1/20 [00:02<00:51,  2.70s/it]

{'epoch': 1, 'loss': 0.979896605014801, 'mae': 0.7438562512397766, 'mse': 0.8789100646972656}


Fitting VAE model on dataset...:  10%|██████▌                                                           | 2/20 [00:05<00:45,  2.54s/it]

{'epoch': 2, 'loss': 1.1313165426254272, 'mae': 0.7441172003746033, 'mse': 0.8786981105804443}


Fitting VAE model on dataset...:  15%|█████████▉                                                        | 3/20 [00:07<00:42,  2.48s/it]

...

{'epoch': 18, 'loss': 0.8790731430053711, 'mae': 0.7436665892601013, 'mse': 0.8789464235305786}


Fitting VAE model on dataset...:  95%|█████████████████████████████████████████████████████████████▊   | 19/20 [00:53<00:02,  2.79s/it]

{'epoch': 19, 'loss': 0.8791211247444153, 'mae': 0.7438399791717529, 'mse': 0.879004180431366}


Fitting VAE model on dataset...: 100%|█████████████████████████████████████████████████████████████████| 20/20 [00:56<00:00,  2.80s/it]

{'epoch': 20, 'loss': 0.879841685295105, 'mae': 0.743800699710846, 'mse': 0.8797286152839661}


In [350]:
rep = r.latent_rep()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 26.24it/s]


(26, 1000)